In [1]:
import src.triangle_ray_intersection as triangle_ray_intersection

In [8]:
names = ['thorax', 'neck', 'a_L1', 'a_L2', 'a_R1', 'a_R2', 'leg_f_L1', 'leg_f_L2', 'leg_f_R1', 'leg_f_R2', 'leg_m_L1', 'leg_m_L2', 'leg_m_R1', 'leg_m_R2', 'a_R0', 'a_L0', 'leg_f_R0', 'leg_f_L0', 'leg_m_R0', 'leg_m_L0', 'eye_L', 'eye_R', 'm_L0', 'm_R0', 'm_L1', 'm_R1']

In [2]:
t = triangle_ray_intersection.CollisionDetector(obj_folder="./example_obj_folder", skeleton_json_path="", pose_csv="example_skeleton_csv.csv")

Frame index is 244 


In [5]:
t._obj_dict[244]

<trimesh.Trimesh(vertices.shape=(2768, 3), faces.shape=(5528, 3), name=`0244.obj`)>

In [9]:
for i in range(2400):
    location = t.get_collisions(i)
    if len(location) > 0:
        # print(location)
        print(i)

736
738
2290
2291
2292
2293
2295
2296
2297
2298
2299
2300
2301
2302
2303
2304
2305
2306
2307
2308
2309
2310
2311
2312
2313
2315
2316
2319
2320
2322
2379
2380
2381
2382
2390
2391
2392
2396
2399
